# LANL Earhquake prediction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:,.10f}'.format
pd.options.display.max_rows = 4000

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# Visualizations
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
sns.set()

In [3]:
FEATURES_BASE = '../data/processed/train/features_base.csv'
FEATURES_BASE_DENOISE = '../data/processed/train/features_base_denoise.csv'
FEATURES_FOLDS_DENOISE = '../data/processed/train/features_folds_denoise.csv'
FEATURES_TSFRESH = '../data/processed/train/features_tsfresh.csv'

In [4]:
%%time
features_base = pd.read_csv(FEATURES_BASE,index_col='id').add_prefix('b_')
features_base_denoise = pd.read_csv(FEATURES_BASE_DENOISE,index_col='id').add_prefix('bd_')
features_folds_denoise = pd.read_csv(FEATURES_FOLDS_DENOISE,index_col='id').add_prefix('fd_')
features_tsfresh = pd.read_csv(FEATURES_TSFRESH,index_col='id').add_prefix('ts_')

CPU times: user 1.74 s, sys: 107 ms, total: 1.85 s
Wall time: 1.84 s


In [5]:
y_all = features_base['b_y']
df_all = features_base.join(features_base_denoise).join(features_folds_denoise).join(features_tsfresh)
df = df_all.drop(['bd_y','fd_y','ts_y'],axis=1)
X_all = df_all.drop(['b_y','bd_y','fd_y','ts_y'],axis=1)

In [6]:
print('features_base.shape:',features_base.shape)
print('features_base_denoise.shape:',features_base_denoise.shape)
print('features_folds_denoise.shape:',features_folds_denoise.shape)
print('features_tsfresh.shape:',features_tsfresh.shape)
print('df_all.shape:',df_all.shape)
print('df.shape:',df.shape)
print('X_all.shape:',X_all.shape)

features_base.shape: (4194, 116)
features_base_denoise.shape: (4194, 116)
features_folds_denoise.shape: (4194, 1151)
features_tsfresh.shape: (4194, 789)
df_all.shape: (4194, 2172)
df.shape: (4194, 2169)
X_all.shape: (4194, 2168)


## Cleaning from NaN,infinity or too large values

In [7]:
%%time
if np.any(np.isnan(X_all)):
    X_all.fillna(0,inplace=True)

CPU times: user 17.7 ms, sys: 411 µs, total: 18.1 ms
Wall time: 16.8 ms


In [8]:
%%time
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = df.corr()

CPU times: user 31.1 s, sys: 18.9 ms, total: 31.1 s
Wall time: 31.1 s


In [ ]:
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [20]:
#Correlation with output variable
cor_target = abs(cor['b_y'])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.4]

In [21]:
relevant_features.index.values

array(['b_q90', 'b_q95', 'b_q05', 'b_q10', 'b_q01_roll_std_10',
       'b_q05_roll_std_10', 'b_q10_roll_std_10', 'b_q95_roll_std_10',
       'b_q05_roll_mean_10', 'b_q95_roll_mean_10', 'b_q01_roll_std_50',
       'b_q05_roll_std_50', 'b_q10_roll_std_50', 'b_q95_roll_std_50',
       'b_min_roll_std_100', 'b_q01_roll_std_100', 'b_q05_roll_std_100',
       'b_q10_roll_std_100', 'b_q95_roll_std_100', 'b_min_roll_std_1000',
       'b_q01_roll_std_1000', 'b_q05_roll_std_1000',
       'b_q10_roll_std_1000', 'b_q95_roll_std_1000', 'b_y', 'bd_q95',
       'bd_q05', 'bd_q95_roll_std_10', 'bd_q05_roll_mean_10',
       'bd_q95_roll_mean_10', 'bd_q01_roll_abs_mean_10',
       'bd_q95_roll_abs_mean_10', 'bd_q95_roll_std_50',
       'bd_q95_roll_std_100', 'bd_q95_roll_std_1000',
       'fd_f_0_q01_roll_abs_mean_10', 'fd_f_1_q01_roll_abs_mean_10',
       'fd_f_2_q01_roll_abs_mean_10', 'fd_f_3_q01_roll_abs_mean_10',
       'fd_f_4_q01_roll_abs_mean_10', 'fd_f_5_q01_roll_abs_mean_10',
       'fd_f_6_q01

## Outer Cross-Validation split

In [ ]:
from sklearn.model_selection import train_test_split
from src.config.common import RANDOM_STATE
X, X_cross, y, y_cross = train_test_split(X_all, y_all, test_size=0.15, random_state=RANDOM_STATE)

## Feature selection

In [ ]:
from src.models.grid import grid_search as gs
from src.config.grid import GridSpace

In [ ]:
from sklearn.linear_model import ARDRegression,BayesianRidge,ElasticNet,ElasticNetCV
from sklearn.linear_model import HuberRegressor, Lars,LarsCV,Lasso,LassoCV,LassoLars,LassoLarsCV
from sklearn.linear_model import LassoLarsIC,LinearRegression,MultiTaskElasticNet,MultiTaskElasticNetCV
from sklearn.linear_model import MultiTaskLasso, MultiTaskLassoCV, PassiveAggressiveRegressor, RANSACRegressor
from sklearn.linear_model import Ridge, RidgeCV, SGDRegressor, TheilSenRegressor
from sklearn.kernel_ridge import KernelRidge

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3)

In [ ]:
cv_dict = {}

In [ ]:
%%time
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA

i=0
k_features = [50,100,200]
scalers = []

cv_dict['ridge'] = {}
for k_feature in k_features:
    cv_dict['ridge'][k_feature] = {
        'best_params':[],
        'best_score':[],
        'time_left':[],
        'mae':[]
    }
    for train_index, test_index in kf.split(X):
        print(k_feature)
        i+=1
    #     print('kfold.iteration=',i)
        X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

    #     print('VarianceThreshold')
        var_selector = VarianceThreshold().fit(X_train)

    #     print('before.X.shape:',X_train.shape)
        X_train = X_train[X_train.columns[var_selector.get_support(indices=True)]]
    #     print('after.X.shape:',X_train.shape)

    #     print('SelectKBest')
#         pca = PCA(n_components=k_feature)
#         pca = pca.fit(X_train)
        
#         X_train = pd.DataFrame(data=pca.transform(X_train))
        
        
        kbest_selector = SelectKBest(f_regression, k=k_feature)
        kbest_selector = kbest_selector.fit(X_train, y_train)
        X_train = X_train[X_train.columns[kbest_selector.get_support(indices=True)]]

    #     print('Scaler')
#         scaler = StandardScaler()
        scaler = RobustScaler()
        scaler = scaler.fit(X_train)

        X_train_scale = scaler.transform(X_train)

        best_params, best_score, time_left = gs(Ridge(), GridSpace.ridge, X_train_scale,y_train)
        print(best_params,'loss:',best_score,'minutes_left:',time_left,'k_features_count:',k_feature)

    #     test dataset
        
        X_test = X_test[X_test.columns[var_selector.get_support(indices=True)]]
#         X_test = pca.transform(X_test)
        X_test = X_test[X_test.columns[kbest_selector.get_support(indices=True)]]
        X_test_scale = scaler.transform(X_test)

        model = Ridge(**best_params)
        model.fit(X_train_scale,y_train)
        prediction = model.predict(X_test_scale)

        cv_dict['ridge'][k_feature]['best_params'].append(best_params)
        cv_dict['ridge'][k_feature]['best_score'].append(best_score)
        cv_dict['ridge'][k_feature]['time_left'].append(time_left)
        cv_dict['ridge'][k_feature]['mae'].append(mean_absolute_error(y_test, prediction))

In [ ]:
def print_cv(cv_dict):
    for model_name,cv in cv_dict.items():
        for key,val in cv.items():
            print(model_name,key,np.mean(val['mae']))

In [ ]:
print_cv(cv_dict)

In [ ]:
print_cv(cv_dict)

## Trees

In [ ]:
num_folds = 5

def make_predictions(estimator, features, target, test=None, plot=True, lgb=False):
    """Train the estimator and make predictions for oof and test data."""
    folds = KFold(num_folds, shuffle=True, random_state=2019)
    oof_predictions = np.zeros(features.shape[0])
    if test is not None:
        sub_predictions = np.zeros(test.shape[0])
    for (train_index, valid_index) in folds.split(features, target):
        
        if lgb:
            estimator.fit(features[train_index], target[train_index],
                          early_stopping_rounds=100, verbose=False,
                          eval_set=[(features[train_index], target[train_index]),
                                    (features[valid_index], target[valid_index])])
        else:
            estimator.fit(features[train_index], target[train_index])
        oof_predictions[valid_index] = estimator.predict(features[valid_index]).flatten()
        if test is not None:
            sub_predictions += estimator.predict(test).flatten() / num_folds
    
    # Plot out-of-fold predictions vs actual values
    if plot:
        fig, axis = plt.subplots(1, 2, figsize=(12,5))
        ax1, ax2 = axis
        ax1.set_xlabel('actual')
        ax1.set_ylabel('predicted')
        ax2.set_xlabel('train index')
        ax2.set_ylabel('time to failure')
        ax1.scatter(target, oof_predictions, color='brown')
        ax1.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)], color='blue')
        ax2.plot(target, color='blue', label='y_train')
        ax2.plot(oof_predictions, color='orange')
    if test is not None:
        return oof_predictions, sub_predictions
    else:
        return oof_predictions

In [ ]:
fixed_params = {
    'objective': 'regression_l1',
    'boosting': 'gbdt',
    'verbosity': -1,
    'random_seed': 19,
    'n_estimators': 20000,
}

param_grid = {
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'num_leaves': list(range(8, 92, 4)),
    'max_depth': [3, 4, 5, 6, 8, 12, 16, -1],
    'feature_fraction': [0.8, 0.85, 0.9, 0.95, 1],
    'subsample': [0.8, 0.85, 0.9, 0.95, 1],
    'lambda_l1': [0, 0.1, 0.2, 0.4, 0.6, 0.9],
    'lambda_l2': [0, 0.1, 0.2, 0.4, 0.6, 0.9],
    'min_data_in_leaf': [10, 20, 40, 60, 100],
    'min_gain_to_split': [0, 0.001, 0.01, 0.1],
}

In [ ]:
var_selector = VarianceThreshold().fit(X)

#     print('before.X.shape:',X_train.shape)
X_train = X_train[X_train.columns[var_selector.get_support(indices=True)]]
#     print('after.X.shape:',X_train.shape)

#     print('SelectKBest')
#         pca = PCA(n_components=k_feature)
#         pca = pca.fit(X_train)

#         X_train = pd.DataFrame(data=pca.transform(X_train))


kbest_selector = SelectKBest(f_regression, k=k_feature)
kbest_selector = kbest_selector.fit(X_train, y_train)
X_train = X_train[X_train.columns[kbest_selector.get_support(indices=True)]]

#     print('Scaler')
#         scaler = StandardScaler()
scaler = RobustScaler()
scaler = scaler.fit(X_train)

X_train_scale = scaler.transform(X_train)

In [ ]:
import random
import lightgbm as lgb
dataset = lgb.Dataset(X, label=y)  # no need to scale features

best_score = 999
best_params= None
best_nrounds = None

for i in range(500):
    params = {k: random.choice(v) for k, v in param_grid.items()}
    params.update(fixed_params)
    result = lgb.cv(params, dataset, nfold=5, early_stopping_rounds=100,
                    stratified=False)
    
    if result['l1-mean'][-1] < best_score:
        best_score = result['l1-mean'][-1]
        best_params = params
        best_nrounds = len(result['l1-mean'])
        
    print(i,best_score,best_nrounds)

In [ ]:
print("Best mean score: {:.4f}, num rounds: {}".format(best_score, best_nrounds))
print(best_params)
gb_oof = make_predictions(lgb.LGBMRegressor(**best_params), X.values, y, lgb=True)